<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [12]:
import pandas as pd

data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

# print(data.head())

train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]
# 일시를 제외해보기로 함
target = data['평균기온']

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(train, target, test_size = 0.3)

In [4]:
from sklearn.metrics import mean_absolute_error
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

hgb = HistGradientBoostingRegressor()
hgb.fit(train_input, train_target)

train_hgb_pred = hgb.predict(train_input)
test_hgb_pred = hgb.predict(test_input)

train_hgb_mae = mean_absolute_error(train_target, train_hgb_pred)
test_hgb_mae = mean_absolute_error(test_target, test_hgb_pred)

print(train_hgb_mae)
print(test_hgb_mae)

print(hgb.score(train_input, train_target))
print(hgb.score(test_input, test_target))

submission['평균기온'] = train_hgb_pred[:358]
print(submission.head(10))

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.3781478418680901
0.39952108180507157
0.9976721536094902
0.9974870146619979
           일시       평균기온
0  2023-01-01  -1.579007
1  2023-01-02  22.918049
2  2023-01-03  22.131631
3  2023-01-04  16.887715
4  2023-01-05  20.640943
5  2023-01-06  30.384745
6  2023-01-07  16.303455
7  2023-01-08  18.345480
8  2023-01-09  12.900488
9  2023-01-10  18.421391


In [15]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23011 entries, 0 to 23010
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일시      23011 non-null  object 
 1   최고기온    23008 non-null  float64
 2   최저기온    23008 non-null  float64
dtypes: float64(2), object(1)
memory usage: 539.4+ KB
None


In [17]:
# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

# 계절 분류 함수 정의
def classify_season(month):
    if  month == 1 or month == 2 or month == 12:
        return '겨울'
    elif 3 <= month <= 5:
        return '봄'
    elif 6 <= month <= 8:
        return '여름'
    else:
        return '가을'

# '일시' 컬럼에서 월 정보를 추출하여 '계절' 열 추가
data['월'] = data['일시'].dt.month
data['계절'] = data['월'].apply(classify_season)

# 결과 확인
print(data)

              일시  최고기온  최저기온   일교차  강수량  평균습도  평균풍속  일조합    일사합   일조율  평균기온  \
0     1960-01-01   2.2  -5.2   7.4  NaN  68.3   1.7  6.7    NaN   NaN  -1.6   
1     1960-01-02   1.2  -5.6   6.8  0.4  87.7   1.3  0.0    NaN   NaN  -1.9   
2     1960-01-03   8.7  -2.1  10.8  0.0  81.3   3.0  0.0    NaN   NaN   4.0   
3     1960-01-04  10.8   1.2   9.6  0.0  79.7   4.4  2.6    NaN   NaN   7.5   
4     1960-01-05   1.3  -8.2   9.5  NaN  44.0   5.1  8.2    NaN   NaN  -4.6   
...          ...   ...   ...   ...  ...   ...   ...  ...    ...   ...   ...   
23006 2022-12-27   3.3  -7.3  10.6  NaN  69.8   1.8  8.8  10.25  91.7  -2.6   
23007 2022-12-28   0.1  -6.0   6.1  0.1  58.1   2.5  8.7  10.86  90.6  -3.3   
23008 2022-12-29   2.1  -7.8   9.9  0.0  56.3   1.7  9.0  10.88  93.8  -2.9   
23009 2022-12-30   2.3  -4.4   6.7  0.0  65.6   1.9  7.9  10.84  82.3  -1.8   
23010 2022-12-31   2.1  -5.1   7.2  0.0  65.5   1.4  1.1   4.16  11.5  -1.2   

        월  계절  
0       1  겨울  
1       1  겨울  
2  

In [20]:
# 계절별 중앙값 계산
seasonal_medians = data.groupby('계절').transform('median')

# 결측값 대체
data.fillna(seasonal_medians, inplace=True)

print(data)

              일시  최고기온  최저기온   일교차       강수량  평균습도  평균풍속  일조합        일사합  \
0     1960-01-01   2.2  -5.2   7.4  2.136753  68.3   1.7  6.7   7.899426   
1     1960-01-02   1.2  -5.6   6.8  0.400000  87.7   1.3  0.0   7.899426   
2     1960-01-03   8.7  -2.1  10.8  0.000000  81.3   3.0  0.0   7.899426   
3     1960-01-04  10.8   1.2   9.6  0.000000  79.7   4.4  2.6   7.899426   
4     1960-01-05   1.3  -8.2   9.5  2.136753  44.0   5.1  8.2   7.899426   
...          ...   ...   ...   ...       ...   ...   ...  ...        ...   
23006 2022-12-27   3.3  -7.3  10.6  2.136753  69.8   1.8  8.8  10.250000   
23007 2022-12-28   0.1  -6.0   6.1  0.100000  58.1   2.5  8.7  10.860000   
23008 2022-12-29   2.1  -7.8   9.9  0.000000  56.3   1.7  9.0  10.880000   
23009 2022-12-30   2.3  -4.4   6.7  0.000000  65.6   1.9  7.9  10.840000   
23010 2022-12-31   2.1  -5.1   7.2  0.000000  65.5   1.4  1.1   4.160000   

             일조율  평균기온   월  계절  
0      54.020054  -1.6   1  겨울  
1      54.020054  -1.

<ipython-input-20-7e65321ee868>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seasonal_medians = data.groupby('계절').transform('median')


In [21]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]
# 일시를 제외해보기로 함
target = data['평균기온']

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(train, target, test_size = 0.3)

In [24]:
print(len(train_input), len(train_target))
print(len(test_input), len(test_target))

16107 16107
6904 6904


In [26]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(train_input, train_target)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train_input, train_target)

pred_xgb = xgb.predict(train_input)
pred_rf = rf.predict(train_input)

In [29]:
from sklearn.metrics import mean_absolute_error

train_mae_xgb = mean_absolute_error(train_target, pred_xgb)
# test_mae_xgb = mean_absolute_error(test_target, pred_xgb)

train_mae_rf = mean_absolute_error(train_target, pred_rf)
# test_mae_rf = mean_absolute_error(test_target, pred_rf)


print('xgb train:', train_mae_xgb)
# print('xgb test:', test_mae_xgb)
print('rf train:', train_mae_rf)
# print('rf test:', test_mae_rf)

submission['평균기온'] = pred_xgb[:358]
print(submission.head(10))
submission['평균기온'] = pred_rf[:358]
print(submission.head(10))

xgb train: 0.2707367184829587
rf train: 0.1417423480474329
           일시       평균기온
0  2023-01-01  18.083761
1  2023-01-02  20.201134
2  2023-01-03  17.254198
3  2023-01-04  12.661690
4  2023-01-05  -0.326550
5  2023-01-06   1.515783
6  2023-01-07  -0.841269
7  2023-01-08  20.667528
8  2023-01-09  15.418682
9  2023-01-10  23.525269
           일시    평균기온
0  2023-01-01  18.225
1  2023-01-02  20.037
2  2023-01-03  17.123
3  2023-01-04  12.524
4  2023-01-05  -0.626
5  2023-01-06   1.517
6  2023-01-07  -0.826
7  2023-01-08  20.320
8  2023-01-09  15.813
9  2023-01-10  23.495


In [30]:
submission.to_csv("DEAGYUL_7_submission.csv", index=False)

print(submission.head())

           일시    평균기온
0  2023-01-01  18.225
1  2023-01-02  20.037
2  2023-01-03  17.123
3  2023-01-04  12.524
4  2023-01-05  -0.626
